In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader(file_path="./files/challenges/document.txt")
doc = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cache_embedding = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vector_store = FAISS.from_documents(doc, cache_embedding)

retriver = vector_store.as_retriever()


def load_memory(_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
    You are good at summarizing and explaining things you have read. expectially novels and storises. Can you summarize the document i gave to you for me? If you can't do when processing document that I gave to you, please let me know.
    -------
    {document}
    """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


chain = (
    {
        "document": retriver,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result.content)


invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write on the table?")
invoke_chain("Who is Julla?")

The document you provided does not mention anyone named Aaronson or discuss any specific crimes related to that name. It primarily focuses on themes of oppression, psychological manipulation, and the internal struggles of a character who is grappling with the Party's control over thoughts and emotions. If you have a specific context or additional information about Aaronson, please provide it, and I can help you analyze that situation further.
In the document, the character traces "2+2=5" in the dust on the table. This phrase symbolizes the oppressive control of the Party over reality and truth, illustrating the extent to which individuals are forced to accept falsehoods as truths under totalitarian rule.
In the document, "Julia" is a significant character who represents love, rebellion, and personal connection in a dystopian society. She is the romantic partner of the protagonist, who experiences deep feelings for her, especially during moments of despair and oppression. Julia embodies